In [ ]:
!pip install fastFM
!pip install surprise 
!pip install lightfm
!pip install hyperopt
!pip uninstall tensorflow
!pip install tensorflow==1.15
!pip install annoy


## Imports

In [ ]:
import pandas as pd
import math
import numpy as np
import random
from numpy.linalg import inv
from numpy.linalg import multi_dot
import matplotlib.pyplot as plt
import itertools
import warnings
from math import sqrt
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from fastFM import als
from sklearn.metrics import mean_squared_error
from lightfm.datasets import fetch_movielens
from lightfm import LightFM
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
warnings.filterwarnings("ignore")
from scipy import sparse
from lightfm import LightFM
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
from hyperopt import tpe, fmin, hp, Trials, STATUS_OK,space_eval
import sys
import seaborn as sns
from timeit import Timer
from datetime import datetime

from hyperopt import tpe, fmin, hp, Trials, STATUS_OK,space_eval
from collections import defaultdict
import random
from sklearn.metrics import mean_squared_error
from numpy.linalg import inv
import pandas as pd
import numpy as np
import math
from numpy.linalg import inv
from numpy.linalg import multi_dot
import matplotlib.pyplot as plt
import itertools
import warnings
warnings.filterwarnings("ignore")

from annoy import AnnoyIndex


In [ ]:
from ncf_singlenode import *
from dataset import Dataset as NCFDataset
from constants import SEED as DEFAULT_SEED
SEED = DEFAULT_SEED

## Get data

In [ ]:
def get_data():
  !pip install -U -q PyDrive
  from pydrive.auth import GoogleAuth
  from pydrive.drive import GoogleDrive
  from google.colab import auth
  from oauth2client.client import GoogleCredentials
  # Authenticate and create the PyDrive client.
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)

  id = "1FFYnZRIuzQLeBkDUuJpK_oRtUmdxMd9O"

  downloaded = drive.CreateFile({'id':id}) 
  downloaded.GetContentFile('final_dataset.csv')  
  data = pd.read_csv('final_dataset.csv')
  return data

data = get_data()


In [ ]:
def train_test_split(data):

  user_freq=data.groupby(['userId']).size().reset_index(name='counts')
  users_lt3=user_freq[user_freq['counts']<3][['userId']]
  users_ge3=user_freq[user_freq['counts']>=3][['userId']]
  train1=pd.merge(data, users_lt3, on=['userId'],how='inner')
  data1=pd.merge(data, users_ge3, on=['userId'],how='inner')
  data1.sort_values(['userId', 'timestamp'], ascending=[True, False],inplace=True)
  test=data1.groupby('userId').sample(frac=.3, random_state=2) 
  test_idx=data1.index.isin(test.index.to_list())
  train=train1.append(data1[~test_idx])
  return train, test, user_freq

train, test, user_freq = train_test_split(data)



## ANN-NCF model

In [ ]:
def run_ann_ncf_pipeline(train, test, user_freq):
  train_df = train
  test_df = test

  def create_SVD_UV(train,test,user_freq):
    train=train[['userId', 'movieId', 'rating']]
    test=test[['userId', 'movieId', 'rating']]
    reader = Reader(rating_scale=(1,5))
    train = Dataset.load_from_df(train[['userId', 'movieId', 'rating']], reader=reader)
    test = Dataset.load_from_df(test[['userId', 'movieId', 'rating']], reader=reader)
    raw_ratings = test.raw_ratings
    threshold = int(1 * len(raw_ratings))
    A_raw_ratings = raw_ratings[:threshold]
    test = test.construct_testset(A_raw_ratings)

    raw_ratings1 = train.raw_ratings
    threshold = int(1 * len(raw_ratings1))
    B_raw_ratings = raw_ratings1[:threshold]
    train_test = train.construct_testset(B_raw_ratings)

    model = SVD(n_epochs=50,n_factors=15,reg_all=0.1,lr_all=0.02)
    trainset = train.build_full_trainset()
    model.fit(trainset)

    # Retrieiving inner ids, as used by surprise package during model training
    user_inner_ids = [x for x in trainset.all_users()]
    item_inner_ids = [i for i in trainset.all_items()]

    # All ids mapped back to values in the actual train set 
    user_raw_ids = [trainset.to_raw_uid(x) for x in user_inner_ids]
    item_raw_ids = [trainset.to_raw_iid(x) for x in item_inner_ids]

    U = model.pu 
    V = model.qi

    return U, V, trainset, user_inner_ids



  U,V, trainset, user_inner_ids = create_SVD_UV(train,test,user_freq)

  def run_ann(user_inner_ids, V, train_df):
    #To be used for serialized process; user item search

    f = 15 # n_factors
    t = AnnoyIndex(f, 'angular')  # Length of item vector that will be indexed
    for i,v in zip(user_inner_ids,V):
        t.add_item(i, v)

    t.build(100)
    t.save('test_user_item.ann')


    u = AnnoyIndex(f, 'angular')
    u.load('test_user_item.ann') 
    # super fast, will just map the file

    user_ids_nn = list(user_freq[user_freq['num_movies']>50]['userId'].values)

    def find_nn_greater_than_k(user_id, k=100):
      return t.get_nns_by_vector(U[trainset.to_inner_uid(user_id)], k) 

    user_nn_items = list(map(find_nn_greater_than_k,user_ids_nn))
    user_nn_items_dict={}
    for userid,items in zip(user_ids_nn,user_nn_items):
      user_nn_items_dict[userid] = [trainset.to_raw_iid(i) for i in items]
    
    user_nn_items_df = pd.concat({k: pd.Series(v) for k, v in user_nn_items_dict.items()})
    user_nn_items_df = user_nn_items_df.reset_index()
    user_nn_items_df = user_nn_items_df.drop(columns=['level_1'])
    user_nn_items_df.rename(columns={'level_0': 'userId', 0: 'movieId'}, inplace=True)

    merged_df= pd.merge(train_df,user_nn_items_df,how='inner',on=['userId','movieId'])

    #concatenate data for users who rated less than 50 movies merged_df
    user_ids_without_nn = set(list(train_df['userId'].unique())) - set(user_ids_nn)
    user_ids_without_nn_df = pd.DataFrame(user_ids_without_nn,columns =['userId']) 

    user_ids_without_nn_df = pd.merge(train_df,user_ids_without_nn_df,how='inner',on='userId')

    # ANN reduced matrix
    train_concat_df = pd.concat([user_ids_without_nn_df, merged_df])
    #train_concat_df = merged_df
    return train_concat_df


  train_concat_df = def run_ann(user_inner_ids, V, train_df)

  def run_ncf(train_concat_df,test_df):
    train=train_df[['userId' ,'movieId' ,'rating']]
    test=test_df[['userId' ,'movieId' ,'rating']]
    train.rename({'userId': 'userID', 'movieId': 'itemID'}, axis=1, inplace=True)
    test.rename({'userId': 'userID', 'movieId': 'itemID'}, axis=1, inplace=True)
    test=pd.merge(test,pd.DataFrame(np.unique(train['userID']),columns=['userID']),on=['userID'],how='inner')
    data = NCFDataset(train=train, test=test, seed=SEED)
    
    EPOCHS = 100
    BATCH_SIZE = 256

    model = NCF (
        n_users=data.n_users, 
        n_items=data.n_items,
        model_type="NeuMF",
        n_factors=4,
        layer_sizes=[16,8,4],
        n_epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        learning_rate=1e-3,
        verbose=10,
        seed=SEED)
    model.fit(data)

    users, items, preds = [], [], []
    item = list(train.itemID.unique())
    for user in train.userID.unique():
        user = [user] * len(item) 
        users.extend(user)
        items.extend(item)
        preds.extend(list(model.predict(user, item, is_list=True)))

    all_predictions = pd.DataFrame(data={"userID": users, "itemID":items, "prediction":preds})

    merged = pd.merge(train, all_predictions, on=["userID", "itemID"], how="outer")
    all_predictions = merged[merged.rating.isnull()].drop('rating', axis=1)

    return all_predictions

  ann_ncf_predictions = run_ncf(train_concat_df, test_df)
  return ann_ncf_predictions


pipeline_predictions = run_ann_ncf_pipeline(train, test, user_freq)